# PhysioNet Motor Imagery - Complete Model Comparison

## Compare All 7 Models

This notebook loads results from:
- **EEG-ARNN Models**: Baseline, Adaptive Gating
- **Legacy Methods**: FBCSP, CNN-SAE, EEGNet, ACS-SE-CNN, G-CARM

And generates comprehensive comparison tables and visualizations.

## Input Files:
- `eeg_arnn_baseline_results.csv`
- `eeg_arnn_adaptive_results.csv`
- `legacy_fbcsp_results.csv`
- `legacy_cnn_sae_results.csv`
- `legacy_eegnet_results.csv`
- `legacy_acs_se_cnn_results.csv`
- `legacy_g_carm_results.csv`

## Output:
- Comparison tables
- Statistical tests
- Visualizations
- `final_comparison_summary.csv`

## 1. Setup and Imports

In [ ]:
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
sns.set_context('notebook', font_scale=1.2)

print("Libraries loaded!")

## 2. Load All Results

In [ ]:
results_dir = Path('results')

model_files = {
    'Baseline EEG-ARNN': 'eeg_arnn_baseline_results.csv',
    'Adaptive Gating EEG-ARNN': 'eeg_arnn_adaptive_results.csv',
    'FBCSP': 'legacy_fbcsp_results.csv',
    'CNN-SAE': 'legacy_cnn_sae_results.csv',
    'EEGNet': 'legacy_eegnet_results.csv',
    'ACS-SE-CNN': 'legacy_acs_se_cnn_results.csv',
    'G-CARM': 'legacy_g_carm_results.csv'
}

results = {}

for model_name, filename in model_files.items():
    filepath = results_dir / filename
    if filepath.exists():
        results[model_name] = pd.read_csv(filepath)
        print(f"Loaded: {model_name} ({len(results[model_name])} subjects)")
    else:
        print(f"Warning: {filename} not found")

print(f"\nTotal models loaded: {len(results)}")

## 3. Aggregate Statistics

In [ ]:
summary_data = []

for model_name, df in results.items():
    summary_data.append({
        'Model': model_name,
        'Accuracy': f"{df['accuracy'].mean():.4f} ± {df['accuracy'].std():.4f}",
        'Precision': f"{df['precision'].mean():.4f} ± {df['precision'].std():.4f}",
        'Recall': f"{df['recall'].mean():.4f} ± {df['recall'].std():.4f}",
        'F1-Score': f"{df['f1_score'].mean():.4f} ± {df['f1_score'].std():.4f}",
        'AUC-ROC': f"{df['auc_roc'].mean():.4f} ± {df['auc_roc'].std():.4f}",
        'Specificity': f"{df['specificity'].mean():.4f} ± {df['specificity'].std():.4f}",
        'N_subjects': len(df)
    })

summary_df = pd.DataFrame(summary_data)
summary_df = summary_df.sort_values('Model')

print("\n" + "="*120)
print("COMPREHENSIVE MODEL COMPARISON")
print("="*120)
print(summary_df.to_string(index=False))
print("="*120)

## 4. Statistical Comparison

In [ ]:
print("\n" + "="*80)
print("STATISTICAL SIGNIFICANCE TESTS (Paired t-test)")
print("="*80 + "\n")

if 'Adaptive Gating EEG-ARNN' in results and 'Baseline EEG-ARNN' in results:
    adaptive_acc = results['Adaptive Gating EEG-ARNN']['accuracy'].values
    baseline_acc = results['Baseline EEG-ARNN']['accuracy'].values
    
    if len(adaptive_acc) == len(baseline_acc):
        t_stat, p_value = stats.ttest_rel(adaptive_acc, baseline_acc)
        improvement = adaptive_acc.mean() - baseline_acc.mean()
        
        print("Adaptive Gating vs Baseline EEG-ARNN:")
        print(f"  Mean improvement: {improvement:.4f} ({improvement/baseline_acc.mean()*100:.2f}%)")
        print(f"  t-statistic: {t_stat:.4f}")
        print(f"  p-value: {p_value:.4f}")
        print(f"  Significant: {'Yes' if p_value < 0.05 else 'No'} (alpha=0.05)")
        print()

baseline_model = 'Baseline EEG-ARNN'
if baseline_model in results:
    baseline_acc = results[baseline_model]['accuracy'].values
    
    print(f"\nComparison against {baseline_model}:\n")
    
    for model_name, df in results.items():
        if model_name == baseline_model:
            continue
        
        model_acc = df['accuracy'].values
        
        if len(model_acc) == len(baseline_acc):
            t_stat, p_value = stats.ttest_rel(model_acc, baseline_acc)
            improvement = model_acc.mean() - baseline_acc.mean()
            
            print(f"{model_name}:")
            print(f"  Improvement: {improvement:+.4f} ({improvement/baseline_acc.mean()*100:+.2f}%)")
            print(f"  p-value: {p_value:.4f}")
            print(f"  Significant: {'Yes' if p_value < 0.05 else 'No'}")
            print()

## 5. Visualizations

### 5.1 Box Plot Comparison

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
fig.suptitle('Model Performance Comparison Across All Metrics', fontsize=16, fontweight='bold')

metrics = ['accuracy', 'precision', 'recall', 'f1_score', 'auc_roc', 'specificity']
metric_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-ROC', 'Specificity']

for idx, (metric, metric_name) in enumerate(zip(metrics, metric_names)):
    ax = axes[idx // 3, idx % 3]
    
    data_for_plot = []
    labels_for_plot = []
    
    for model_name, df in results.items():
        if metric in df.columns:
            data_for_plot.append(df[metric].values)
            labels_for_plot.append(model_name.replace(' EEG-ARNN', '').replace(' ', '\n'))
    
    bp = ax.boxplot(data_for_plot, labels=labels_for_plot, patch_artist=True)
    
    for patch in bp['boxes']:
        patch.set_facecolor('skyblue')
        patch.set_alpha(0.7)
    
    ax.set_ylabel(metric_name, fontsize=12)
    ax.set_title(f'{metric_name} Distribution', fontsize=12, fontweight='bold')
    ax.tick_params(axis='x', rotation=45)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/model_comparison_boxplots.png', dpi=150, bbox_inches='tight')
plt.show()

print("Box plots saved to results/model_comparison_boxplots.png")

### 5.2 Bar Chart - Mean Performance

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

model_names = list(results.keys())
accuracies = [results[m]['accuracy'].mean() for m in model_names]
std_accs = [results[m]['accuracy'].std() for m in model_names]

x_pos = np.arange(len(model_names))
bars = ax.bar(x_pos, accuracies, yerr=std_accs, capsize=5, alpha=0.8, 
              color=['#2ecc71' if 'Adaptive' in m else '#3498db' if 'Baseline' in m else '#95a5a6' 
                     for m in model_names])

ax.set_xlabel('Model', fontsize=14, fontweight='bold')
ax.set_ylabel('Accuracy', fontsize=14, fontweight='bold')
ax.set_title('Mean Accuracy Comparison Across All Models', fontsize=16, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels([m.replace(' EEG-ARNN', '') for m in model_names], rotation=45, ha='right')
ax.set_ylim([min(accuracies) - 0.05, max(accuracies) + 0.05])
ax.grid(True, alpha=0.3, axis='y')

for i, (acc, std) in enumerate(zip(accuracies, std_accs)):
    ax.text(i, acc + std + 0.01, f'{acc:.3f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig('results/model_comparison_accuracy.png', dpi=150, bbox_inches='tight')
plt.show()

print("Bar chart saved to results/model_comparison_accuracy.png")

### 5.3 Radar Chart - Multi-Metric Comparison

In [ ]:
from math import pi

metrics = ['accuracy', 'precision', 'recall', 'f1_score', 'auc_roc', 'specificity']
metric_labels = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-ROC', 'Specificity']

num_vars = len(metrics)
angles = [n / float(num_vars) * 2 * pi for n in range(num_vars)]
angles += angles[:1]

fig, ax = plt.subplots(figsize=(12, 12), subplot_kw=dict(projection='polar'))

colors = ['#2ecc71', '#3498db', '#e74c3c', '#f39c12', '#9b59b6', '#1abc9c', '#34495e']

for idx, (model_name, df) in enumerate(results.items()):
    values = [df[m].mean() for m in metrics]
    values += values[:1]
    
    ax.plot(angles, values, 'o-', linewidth=2, label=model_name.replace(' EEG-ARNN', ''),
            color=colors[idx % len(colors)])
    ax.fill(angles, values, alpha=0.15, color=colors[idx % len(colors)])

ax.set_xticks(angles[:-1])
ax.set_xticklabels(metric_labels, size=12)
ax.set_ylim(0, 1.0)
ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
ax.set_yticklabels(['0.2', '0.4', '0.6', '0.8', '1.0'], size=10)
ax.grid(True)
ax.set_title('Multi-Metric Performance Comparison', size=16, fontweight='bold', pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=10)

plt.tight_layout()
plt.savefig('results/model_comparison_radar.png', dpi=150, bbox_inches='tight')
plt.show()

print("Radar chart saved to results/model_comparison_radar.png")

## 6. Ranking Table

In [ ]:
ranking_data = []

for model_name, df in results.items():
    ranking_data.append({
        'Model': model_name,
        'Mean_Accuracy': df['accuracy'].mean(),
        'Mean_F1': df['f1_score'].mean(),
        'Mean_AUC_ROC': df['auc_roc'].mean(),
    })

ranking_df = pd.DataFrame(ranking_data)
ranking_df['Overall_Score'] = (
    ranking_df['Mean_Accuracy'] * 0.4 + 
    ranking_df['Mean_F1'] * 0.3 + 
    ranking_df['Mean_AUC_ROC'] * 0.3
)
ranking_df = ranking_df.sort_values('Overall_Score', ascending=False).reset_index(drop=True)
ranking_df['Rank'] = range(1, len(ranking_df) + 1)

print("\n" + "="*100)
print("MODEL RANKING (Overall Score = 0.4*Acc + 0.3*F1 + 0.3*AUC)")
print("="*100)
print(ranking_df[['Rank', 'Model', 'Mean_Accuracy', 'Mean_F1', 'Mean_AUC_ROC', 'Overall_Score']].to_string(index=False))
print("="*100)

## 7. Save Final Summary

In [ ]:
summary_df.to_csv('results/final_comparison_summary.csv', index=False)
ranking_df.to_csv('results/final_ranking.csv', index=False)

print("\nFinal summary saved:")
print("  - results/final_comparison_summary.csv")
print("  - results/final_ranking.csv")

print("\n" + "="*80)
print("COMPARISON COMPLETE!")
print("="*80)